In [39]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks, peak_widths, peak_prominences
from BaselineRemoval import BaselineRemoval

In [40]:
global_dir_path = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data"
#4wks
# dir_path = global_dir_path + '/2023_0809_Breath_EA_4w/Spectra'
# dir_path = global_dir_path + '/2023_1019_Breath_EA_4w/Spectra'
# #5wks
# dir_path = global_dir_path + '/2024_0520_Breath_EA_5w/Spectra'
# dir_path = global_dir_path + '/2024_1105_Breath_EA_5w/Spectra'
# #6wks
dir_path = global_dir_path + '/2024_0116_Breath_EA_6w/Spectra' 
# dir_path = global_dir_path + '/2024_0628_Breath_EA_6w/Spectra' #relapse Tx2

dir_list = sorted(os.listdir(dir_path))
print(dir_list)

['2024_01_16_6wks_5plex_EA2_1_blank.txt', '2024_01_16_6wks_5plex_EA2_1_part_001.txt', '2024_01_16_6wks_5plex_EA2_2_blank.txt', '2024_01_16_6wks_5plex_EA2_2_part_001.txt', '2024_01_16_6wks_5plex_EA2_3_blank.txt', '2024_01_16_6wks_5plex_EA2_3_part_001.txt', '2024_01_16_6wks_5plex_EA2_4_blank.txt', '2024_01_16_6wks_5plex_EA2_4_part_001.txt', '2024_01_16_6wks_5plex_EA2_5_blank.txt', '2024_01_16_6wks_5plex_EA2_5_part_001.txt', '2024_01_16_6wks_5plex_EA4_1_blank.txt', '2024_01_16_6wks_5plex_EA4_1_part_001.txt', '2024_01_16_6wks_5plex_EA4_2_blank.txt', '2024_01_16_6wks_5plex_EA4_2_part_001.txt', '2024_01_16_6wks_5plex_EA4_3_blank.txt', '2024_01_16_6wks_5plex_EA4_3_part_001.txt', '2024_01_16_6wks_5plex_EA4_4_blank.txt', '2024_01_16_6wks_5plex_EA4_4_part_001.txt', '2024_01_16_6wks_5plex_EA4_5_blank.txt', '2024_01_16_6wks_5plex_EA4_5_part_001.txt', '2024_01_16_6wks_5plex_EA5_4_blank.txt', '2024_01_16_6wks_5plex_EA5_4_part_001.txt', '2024_01_16_6wks_5plex_EA5_5_blank.txt', '2024_01_16_6wks_5plex_

In [41]:
def get_prominence(d):
    peaks, _ = find_peaks(signal_corr, distance=d, height=HEIGHT, width=1.2) #change, add width
    prominences = peak_prominences(signal_corr, peaks)[0]

    dictionary = {}
    for i in prominences:
        if int(i) not in dictionary:
            dictionary[int(i)] = 1
        else: 
            dictionary[int(i)] += 1 

    result = max(dictionary.values())
    prom = []
    for i in dictionary:
        if dictionary[i] == result:
            prom.append(i)

    best_p = max(prom)

    return best_p

In [42]:
def view_peak(d):
    peaks, _ = find_peaks(signal_corr, distance=d, height=HEIGHT, width=1.2) #change, add width
    plt.plot(m_to_z, signal_corr, label='data (baseline corrected)')
    plt.plot(m_to_z[peaks], signal_corr[peaks], "x", label='peaks')
    plt.xlabel('m/z')
    plt.ylabel('Intensity (a.u.)')
    plt.ylim(0, 200)
    plt.xlim(14,300)
    plt.legend()
    plt.show()
    
def get_mz(d):
    peaks, _ = find_peaks(signal_corr, distance=d, height=HEIGHT, width=1.2) #change, add width
    mz = np.zeros(len(peaks))
    for i in range(len(peaks)):
        index = peaks[i]
        mz[i] = m_to_z[index]
    return mz

def min_dist(arr):
    min_dist = 100000
    for i in range(len(arr) - 1):
        if arr[i + 1] - arr[i] < min_dist:
            min_dist = arr[i + 1] - arr[i]
    return min_dist

In [44]:
#Background substraction
for i in range(0, len(dir_list), 2):
    if ".txt" in dir_list[i]:
        name = dir_path + '/' + dir_list[i]
        print(name) # to make sure the bk are the blank files using this method

        bk = pd.read_csv(name, sep='\t')
        df = pd.read_csv(dir_path + '/' + dir_list[i+1], sep='\t')
        
        m_to_z = df.iloc[:,2]
        signal = df.iloc[:,3]
        bk_signal = bk.iloc[:,3]

        sub_signal = signal-bk_signal #removes background signal
        sub_signal = sub_signal.clip(lower=0) #removes negatives
        base=BaselineRemoval(sub_signal)
        signal_corr=base.ZhangFit() #ensures peaks all have the same baseline
        HEIGHT = 10 #change

        #Find min_d
        arr = []
        for d in range(1, 300): #change
            arr.append((d, min_dist(get_mz(d))))
        arr.sort(key = lambda x: x[1])

        for j in range(len(arr)-1):
            if arr[j][1] < 0.05 and arr[j][1] > 0.01: #change 
                min_d = arr[j][0]
                break
        
        peaks, _ = find_peaks(signal_corr, distance=min_d, height=HEIGHT, width=1.2, prominence=get_prominence(min_d))
        
        peak_m_z = [m_to_z[peak] for peak in peaks] 
        intarray = [signal_corr[peak] for peak in peaks] 
             
        zip_result = zip(peak_m_z, intarray)
        columns=['m/z', 'Intensity (corr)']
        df1 = pd.DataFrame(zip_result, columns=columns)

        aucfilename = dir_path + '/Peaks/' + dir_list[i+1] 

        aucfilename = aucfilename.replace(".txt","_peaks.xlsx") 
        df1.to_excel(aucfilename)

C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_0116_Breath_EA_6w/Spectra/2024_01_16_6wks_5plex_EA2_1_blank.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_0116_Breath_EA_6w/Spectra/2024_01_16_6wks_5plex_EA2_2_blank.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_0116_Breath_EA_6w/Spectra/2024_01_16_6wks_5plex_EA2_3_blank.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_0116_Breath_EA_6w/Spectra/2024_01_16_6wks_5plex_EA2_4_blank.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_0116_Breath_EA_6w/Spectra/2024_01_16_6wks_5plex_EA2_5_blank.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_0116_Breath_EA_6w/Spectra/2024_01_16_6wks_5plex_EA4_1_blank.txt
C://Users/chris/OneDrive/Desktop/M